In [7]:
import googleapiclient.discovery
import spacy
from spacy_langdetect import LanguageDetector
import pandas as pd
from spacy.language import Language
import re
from spacytextblob.spacytextblob import SpacyTextBlob
from textblob import TextBlob
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import emoji

import environ

from utils.def_api_google import obtenir_infos_video_et_commentaires
from utils.def_utils import get_lang_detector, comment_analysis, word_cloud, polarity_plot, subjectivity_plot, polarity_on_vpn


In [8]:
nlp = spacy.load("fr_core_news_sm")
nlp.add_pipe("spacytextblob", last=True)
get_lang_detector(nlp, "language_detector")
nlp.add_pipe("language_detector")

In [2]:
env = environ.Env()
environ.Env().read_env('.env')

API_KEY = env('API_KEY')

In [ ]:
videos_youtubeur_1 = ['woiFO4o3CXI',
'7_60w7T1Oak',
'coWT9YsAAC0',
'L9HzH8v0Now',
'N-TCJquxeFk',
'qSUE11N1GPM',
'8jluurK-E6U',
'LyDJTp4nS3w',
'LBraVzyYDKg',
'tCIIrPEF8Hk']

videos_youtubeur_2 = ['bbetYpYH-gI',
'iU34s9rbPoE',
'x2WASA8bb-I',
'JbiBkKy4NFs',
'GFnzjrGh3Yc',
'XhbECIm1Wa4',
'rKRAkNc4OKw',
'7a0DayJnGMc',
'V9dJEWTjxlM',
'dcTAk9ASfIk']

In [ ]:
# On récupère les commentaires des vidéos
for i in videos_youtubeur_1:
    commentaires_1, infos_video_1 = obtenir_infos_video_et_commentaires(i, API_KEY)  
    print(i)

for i in videos_youtubeur_2:
    commentaires_2, infos_video_2 = obtenir_infos_video_et_commentaires(i, API_KEY)
    print(i)

bbetYpYH-gI
iU34s9rbPoE
x2WASA8bb-I
JbiBkKy4NFs
GFnzjrGh3Yc
XhbECIm1Wa4
rKRAkNc4OKw
7a0DayJnGMc
V9dJEWTjxlM
dcTAk9ASfIk


In [ ]:
for i in videos_youtubeur_1:
    print(i)
    with open(f"data/comments/{i}.txt", "r", encoding="utf-8") as f:
                commentaires = [line.strip() for line in f.readlines()]
    commentaires = [c for c in commentaires if c]  # Filter out empty strings and None values
    df_commentaires = pd.DataFrame([{'comment': c} for c in commentaires])
    df = comment_analysis(df_commentaires,nlp,i)

woiFO4o3CXI
7_60w7T1Oak
